In [5]:
from myBdd import *

In [12]:
f = boolfunc.Expression(
    r"input\1.pcn")
print(f)
ordering = [1,2,3,4]
print(ordering)
a = BDD(f, ordering)
a

4
3
3 1 2 3
2 -2 4
2 -3 4
[1, 2, 3, 4]


ValueError: invalid ordering list

In [11]:
a.node.getKey()

(2, <myBdd.BDDNode at 0x2957d5dcc70>, <myBdd.BDDNode at 0x2957d5dcdc0>)

In [9]:
ordering = [1,4,2,3]
b = BDD(f, ordering)
b

ValueError: invalid ordering list